In [1]:
import pandas as pd
import numpy as np
import multiprocessing
import re
import dask
import dask.dataframe as dd
import os
from bs4 import BeautifulSoup
import Levenshtein
import ast

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500
pd.options.display.max_colwidth = 400

# multiprocessing.set_start_method('spawn')  # Required for Mac M1

# Match by name strings

## 0. Define functions

In [2]:
# This function capitalize & remove special characters from a firm name
def proc_name(name):

    name = name.upper()
    name = name.replace('É','E')
    name = name.replace('é','e')
    name = name.replace('','')
    name = name.replace('.','')
    name = name.replace('/','')
    name = name.replace(',','')
    name = name.replace('\'','')
    name = name.replace('-',' ')
    name = name.replace('&AMP;','&')
    name = name.replace('AND;','&')
    name = name.strip()

    return name
    
# This function removes business type from the name string
# It reads in a all-capitalized firm name
def to_essence(name):

    if name[-24:] == ' FOODS INTERNATIONAL INC':
        name = name[:-24]
    if name[-21:] == ' WORLD TRADING CO INC':
        name = name[:-21]
    if name[-18:] == ' (PRIVATE) LIMITED':
        name = name[:-18]
    if name[-18:] == ' UNITED STATES INC':
        name = name[:-18]
    if name[-18:] == ' NORTH AMERICA INC':
        name = name[:-18]
    if name[-18:] == ' INTERNATIONAL LTD':
        name = name[:-18]
    if name[-18:] == ' INTERNATIONAL INC':
        name = name[:-18]
    if name[-17:] == ' INTERNATIONAL BV':
        name = name[:-17]
    if name[-16:]==' ENTERPRISES INC':
        name = name[:-16]
    if name[-14:]==' FOODS COMPANY':
        name = name[:-14]
    if name[-13:]==' FOOD COMPANY':
        name = name[:-13]
    if name[-13:]==' FOODS CO LLC':
        name = name[:-13]
    if name[-13:]==' INCORPORATED':
        name = name[:-13]
    if name[-13:]==' HOLDINGS INC':
        name = name[:-13]    
    if name[-12:]==' COMPANY LTD':
        name = name[:-12]    
    if name[-12:]==' COMPANY LLC':
        name = name[:-12]    
    if name[-12:]==' COMPANY INC':
        name = name[:-12]    
    if name[-12:]==' CORPORATION':
        name = name[:-12]
    if name[-12:]==' AMERICA INC':
        name = name[:-12]
    if name[-12:]==' FOODS GROUP':
        name = name[:-12]
    if name[-12:]==' FOOD GROUP':
        name = name[:-12]
    if name[-11:]==' FOODS CORP':
        name = name[:-11]
    if name[-10:]==' GROUP INC':
        name = name[:-10]
    if name[-10:]==' FOODS INC':
        name = name[:-10]
    if name[-10:]==' FOOD CORP':
        name = name[:-10]
    if name[-10:]==' (PTY) LTD':
        name = name[:-10]
    if name[-9:]==' FOOD INC':
        name = name[:-9]
    if name[-9:]==' FOODS CO':
        name = name[:-9]
    if name[-9:]==' BOTTLING':
        name = name[:-9]
    if name[-8:]==' FOOD CO':
        name = name[:-8]
    if name[-8:]==' PVT LTD':
        name = name[:-8]
    if name[-8:]==' PTE LTD':
        name = name[:-8]
    if name[-8:]==' USA INC':
        name = name[:-8]
    if name[-8:]==' COMPANY':
        name = name[:-8]
    if name[-7:]==' CO LTD':
        name = name[:-7]
    if name[-7:]==' CO LLC':
        name = name[:-7]
    if name[-7:]==' CENTER':
        name = name[:-7]
    if name[-7:]==' CO INC':
        name = name[:-7]
    if name[-7:]==' US INC':
        name = name[:-7]
    if name[-6:]==' (THE)':
        name = name[:-6]
    if name[-6:]==' MEATS':
        name = name[:-6]
    if name[-6:]==' (USA)':
        name = name[:-6]
    if name[-6:]==' BTTLG':
        name = name[:-6]
    if name[-5:]==' CORP':
        name = name[:-5]
    if name[-5:]==' MEAT':
        name = name[:-5]
    if name[-4:]==' INC':
        name = name[:-4]
    if name[-4:]==' LLC':
        name = name[:-4]
    if name[-4:]==' LLP':
        name = name[:-4]
    if name[-4:]==' CTR':
        name = name[:-4]
    if name[-4:]==' LTD':
        name = name[:-4]
    if name[-4:]==' USA':
        name = name[:-4]
    if name[-3:]==' AG':
        name = name[:-3]
    if name[-3:]==' CO':
        name = name[:-3]
    if name[-3:]==' LP':
        name = name[:-3]
    if name[-3:]==' SE':
        name = name[:-3]
    if name[-3:]==' LC':
        name = name[:-3]
    if name[:4] == 'THE ':
        name = name[4:]

    return name


## 1. Clean BoardEx company name

In [3]:
# Import BoardEx data

boardexemployment = pd.read_stata("../../RawData/boardexemployment_new.dta")
boardexemployment.columns= boardexemployment.columns.str.lower()
boardexemployment = boardexemployment[boardexemployment["hocountryname"]=="United States"]
BoardEx_nameid = boardexemployment.groupby(['companyname', 'companyid']).size()
BoardEx_nameid = BoardEx_nameid.reset_index()

In [4]:
# Clean BoardEx names

def proc_list(BoardEx_nameid):

    output = []

    for index, row in BoardEx_nameid[:].iterrows():

        companyname = row['companyname']
        
        companyname = companyname.upper()

        # If some notes are next to name, remove it

        if re.search("(.*) [(]DE-LISTED (.*)[)]", companyname) is not None:
            companyname = re.search('(.*) [(]DE-LISTED (.*)[)]', companyname).group(1)
        elif re.search('(.*) [(]LISTED (.*)[)]', companyname) is not None:
            companyname = re.search('(.*) [(]LISTED (.*)[)]', companyname).group(1)
        elif re.search('(.*) [(]CEASED TRADING (.*)[)]', companyname) is not None:
            companyname = re.search('(.*) [(]CEASED TRADING (.*)[)]', companyname).group(1)

        # Parse raw company name

        if re.search('(.*) [(]FORMERLY KNOWN AS (.*)\)', companyname)!=None:
            names = re.search('(.*) \(FORMERLY KNOWN AS (.*)\)', companyname)
            name1 = names.group(1)
            name2 = names.group(2)
            names = [name1,name2]
        elif re.search('(.*) \( FORMERLY KNOWN AS (.*)\)', companyname)!=None:
            names = re.search('(.*) \( FORMERLY KNOWN AS (.*)\)', companyname)
            name1 = names.group(1)
            name2 = names.group(2)
            names = [name1,name2]
        elif re.search('(.*) \((.*) PRIOR TO', companyname)!=None:
            names = re.search('(.*) \((.*) PRIOR TO', companyname)
            name1 = names.group(1)
            name2 = names.group(2)
            names = [name1,name2]
        elif re.search('(.*) \((.*)\)', companyname)!=None:
            names = re.search('(.*) \((.*)\)', companyname)
            name1 = names.group(1)
            name2 = names.group(2)
            if len(name2) > 4:  # Abbreviations with less than 5 may be wrong matches
                names = [name1, name2]
            else:
                names = [name1]
        else:
            names = [companyname]

        # Process refined company name

        for name in names:

            name = proc_name(name)
            name = to_essence(name)

            if name=="":
                continue

            output = output+[{
                'BoardExname':row['companyname'],
                'clean_name':name,
                'companyid':row['companyid']
            }]

    # Export results 
    output = pd.DataFrame(output,columns=['BoardExname','clean_name','companyid'])

    return output

# Process parallelly

BoardEx_nameid_dd = dd.from_pandas(BoardEx_nameid, npartitions=20)
with dask.config.set(scheduler='processes'):
    BoardEx_nameid = BoardEx_nameid_dd.map_partitions(proc_list, meta=pd.DataFrame(columns=['BoardExname','clean_name','companyid'])).compute()

BoardEx_nameid = BoardEx_nameid.drop_duplicates(['clean_name', 'companyid'])

In [5]:
# Out of 899,289 raw names that appear BoardEx, 853,265 correspond to a unique cleaned name.
# For the rest, there are many cleaned names that correspond to the same cleaned name.
# For now, I keep the entry with a smaller companyid

BoardEx_nameid.sort_values(["clean_name","companyid"])
BoardEx_nameid = BoardEx_nameid.drop_duplicates(['clean_name'], keep='first')

## 2. Clean Nielsen company name

### Import Nielsen names

In [6]:
Nielsen_nameprefix = pd.read_pickle("../CleanedData/Nielsen_nameprefix.pickle")

### Parse & Clean company name

In [7]:
output = []

for index, row in Nielsen_nameprefix.iterrows():

    companyname = row['Company']
    brcount = companyname.count("<br/>") # Range from 2 to 5

    if brcount==2:
        test = re.search('<td class="dt-Company-field sorting_1">(.*?)<br/>(.*?)<br/>(.*?)</td>', companyname)
        prefix = row['prefix']
        name = test.group(1)
        add1 = test.group(2)
        add2 = test.group(3)
        output = output+[{'Nielsenname':companyname,'prefix':prefix,'name':name,'add1':add1,'add2':add2}]
        (test,prefix,name,add1,add2) = (None,None,None,None,None)
    elif brcount==3:
        test = re.search('<td class="dt-Company-field sorting_1">(.*?)<br/>(.*?)<br/>(.*?)<br/>(.*?)</td>', companyname)
        prefix = row['prefix']
        name = test.group(1)
        add1 = test.group(2)
        add2 = test.group(3)
        add3 = test.group(4)
        output = output+[{'Nielsenname':companyname,'prefix':prefix,'name':name,'add1':add1,'add2':add2,'add3':add3}]
        (test,prefix,name,add1,add2,add3) = (None,None,None,None,None,None)
    elif brcount==4:
        test = re.search('<td class="dt-Company-field sorting_1">(.*?)<br/>(.*?)<br/>(.*?)<br/>(.*?)<br/>(.*?)</td>', companyname)
        prefix = row['prefix']
        name = test.group(1)
        add1 = test.group(2)
        add2 = test.group(3)
        add3 = test.group(4)
        add4 = test.group(5)
        output = output+[{'Nielsenname':companyname,'prefix':prefix,'name':name,'add1':add1,'add2':add2,'add3':add3,'add4':add4}]
        (test,prefix,name,add1,add2,add3,add4) = (None,None,None,None,None,None,None)
    elif brcount==5:
        test = re.search('<td class="dt-Company-field sorting_1">(.*?)<br/>(.*?)<br/>(.*?)<br/>(.*?)<br/>(.*?)<br/>(.*?)</td>', companyname)
        prefix = row['prefix']
        name = test.group(1)
        add1 = test.group(2)
        add2 = test.group(3)
        add3 = test.group(4)
        add4 = test.group(5)
        add5 = test.group(6)
        output = output+[{'Nielsenname':companyname,'prefix':prefix,'name':name,'add1':add1,'add2':add2,'add3':add3,'add4':add4,'add5':add5}]
        (test,prefix,name,add1,add2,add3,add4,add5) = (None,None,None,None,None,None,None,None)

Nielsen_nameprefix = pd.DataFrame(output)

# Clean company name

output = []

for index, row in Nielsen_nameprefix.iterrows():

    name = row['name']
    name = proc_name(name)
    name = to_essence(name)

    output = output+\
    [{'Nielsenname':row['Nielsenname'],'clean_name':name,'prefix':row['prefix']}]

Nielsen_nameprefix = pd.DataFrame(output)

## 3. Match BoardEx to Nielsen

In [8]:
# Exact Merge based on name
Merged = pd.merge(Nielsen_nameprefix, BoardEx_nameid, on="clean_name", how="outer")

# Names on Nielsen that can be exact-matched to BoardEx
Merged_usingparent = Merged[(Merged['BoardExname'].notnull()) & (Merged['Nielsenname'].notnull())]

# Names on Nielsen that cannot be exact-matched to BoardEx
Nielsen_not_merged = Merged[Merged['BoardExname'].isnull() & (Merged['Nielsenname'].notnull())]

## 4. Match on subsidiary names

### Match subsidiary to BoardEx 

In [9]:
# Import subsidiary data

wrdsapps_subsidiary = pd.read_stata("../../RawData/WRDS/wrdsapps_subsidiary.dta")
wrdsapps_subsidiary = wrdsapps_subsidiary[wrdsapps_subsidiary["country_code"]=="US"]
wrdsapps_subsidiary = wrdsapps_subsidiary[["gvkey","coname","clean_company"]]  # clean_company is name of subsidiary
wrdsapps_subsidiary = wrdsapps_subsidiary.drop_duplicates(["coname","clean_company"])

In [10]:
# Assign every subsidiary a GVKEY of its parent

# First, if companyid available, match by it

gvkeycompanyid = pd.read_stata("../CleanedData/gvkeycompanyid.dta")
wrdsapps_subsidiary_wGVKEY = wrdsapps_subsidiary[wrdsapps_subsidiary["gvkey"]!=""]
wrdsapps_subsidiary_wGVKEY = wrdsapps_subsidiary_wGVKEY.rename(columns={"gvkey": "GVKEY"})
wrdsapps_subsidiary_wGVKEY = pd.merge(wrdsapps_subsidiary_wGVKEY, gvkeycompanyid, on="GVKEY", how="inner")

wrdsapps_subsidiary_wGVKEY = wrdsapps_subsidiary_wGVKEY.rename(columns={"clean_company":"subsidiary","coname":"parent_raw"})
wrdsapps_subsidiary_wGVKEY = wrdsapps_subsidiary_wGVKEY[["parent_raw","companyid","subsidiary"]]

In [11]:
# Second, if GVKEY not avaiable, match by name

wrdsapps_subsidiary_noGVKEY = wrdsapps_subsidiary[wrdsapps_subsidiary["gvkey"]==""]

# Clean name in WRDS subsidiary dataset (those without GVKEY)

def proc_list(wrdsapps_subsidiary_noGVKEY):

    output = []
    
    for index, row in wrdsapps_subsidiary_noGVKEY.iterrows():

        name = row['coname']
        name = proc_name(name)
        name = to_essence(name)

        output = output+[{'parent_raw':row['coname'],'parent_cleaned':name,'subsidiary':row['clean_company']}]

    output = pd.DataFrame(output)

    return output

wrdsapps_subsidiary_noGVKEY_dd = dd.from_pandas(wrdsapps_subsidiary_noGVKEY, npartitions=20)
with dask.config.set(scheduler='processes'):
    wrdsapps_subsidiary_noGVKEY = wrdsapps_subsidiary_noGVKEY_dd.map_partitions(proc_list, meta=pd.DataFrame(columns=['parent_raw','parent_cleaned','subsidiary'])).compute()

# Merge WRDS subsidiary dataset (those without GVKEY) to BoardEx by parent name
wrdsapps_subsidiary_noGVKEY = wrdsapps_subsidiary_noGVKEY.rename(columns={"parent_cleaned":"clean_name"})
wrdsapps_subsidiary_noGVKEY = pd.merge(wrdsapps_subsidiary_noGVKEY, BoardEx_nameid, on="clean_name", how="inner")
wrdsapps_subsidiary_noGVKEY = wrdsapps_subsidiary_noGVKEY.rename(columns={"clean_name":"parent_cleaned"})
wrdsapps_subsidiary_noGVKEY = wrdsapps_subsidiary_noGVKEY[["parent_raw","companyid","subsidiary"]]

In [12]:
# Combine two parts

wrdsapps_subsidiary = pd.concat([wrdsapps_subsidiary_wGVKEY,wrdsapps_subsidiary_noGVKEY])
wrdsapps_subsidiary

,parent_raw,companyid,subsidiary
0,K TRON INTERNATIONAL INC,17625.0,K Tron America Inc . . . . . . . . . . . . .
1,K TRON INTERNATIONAL INC,17625.0,K Tron Investment Co . . . . . . . . . . . . . . . . . .
2,K TRON INTERNATIONAL INC,17625.0,K Tron Technologies Inc . . . . . . . . . . . . . . . .
3,K TRON INTERNATIONAL INC,17625.0,K Tron America Inc
4,K TRON INTERNATIONAL INC,17625.0,K Tron Investment Co
...,...,...,...
104590,LETICA CORP,878063.0,Providencia USA Inc
104591,LETICA CORP,878063.0,RPC Packaging Holdings (US) Inc
104592,"FIBERWEB, LLC",633777.0,Dominion Textile (USA) LLC
104593,"FIBERWEB, LLC",633777.0,Providencia USA Inc


### Clean subsidiary names

In [13]:
%%time

def proc_list(wrdsapps_subsidiary):

    output = []
    for index, row in wrdsapps_subsidiary.iterrows():
        name = row['subsidiary']
        name = proc_name(name)
        name = to_essence(name)
        output = output+[{'parent_raw':row['parent_raw'],'companyid':row['companyid'],'subsidiary':name}]
    output = pd.DataFrame(output)
    return output
    
wrdsapps_subsidiary_dd = dd.from_pandas(wrdsapps_subsidiary, npartitions=20)
with dask.config.set(scheduler='processes'):
    wrdsapps_subsidiary = wrdsapps_subsidiary_dd.map_partitions(proc_list, meta=pd.DataFrame(columns=['parent_raw','companyid','subsidiary'])).compute()


CPU times: user 419 ms, sys: 139 ms, total: 558 ms
Wall time: 14.3 s


In [14]:
# Different subsidiary might end up having the same name after cleaning.
# For now, randomly keep one

wrdsapps_subsidiary = wrdsapps_subsidiary.drop_duplicates(["subsidiary"])

### Match remaining names in Nielsen to subsidiaries

In [15]:
wrdsapps_subsidiary = wrdsapps_subsidiary.rename(columns={"subsidiary":"clean_name"})
Nielsen_not_merged = Nielsen_not_merged[["Nielsenname","prefix","clean_name"]]
Merged = pd.merge(wrdsapps_subsidiary, Nielsen_not_merged, on="clean_name", how="outer")
Merged = Merged.rename(columns={"parent_raw":"BoardExname"})

# Names on Nielsen that cannot be exact-matched to BoardEx
Nielsen_not_merged = Merged[Merged['BoardExname'].isnull() & (Merged['Nielsenname'].notnull())]

# Names on Nielsen that can be exact-matched to subsidiary of a parent firm in BoardEx
Merged_usingsubsidiary = Merged[(Merged['BoardExname'].notnull()) & (Merged['Nielsenname'].notnull())]
Merged_usingsubsidiary = Merged_usingsubsidiary[["Nielsenname","clean_name","prefix","BoardExname","companyid"]]

## 5. For those that cannot be exact-matched to a firm in BoardEx or a subsidiary, Do fuzzy matching

In [16]:
# # Note that using L-distance is superior to using word-as-vector representation for firm name matching

# # See: https://stackoverflow.com/questions/13636848/is-it-possible-to-do-fuzzy-match-merge-with-python-pandas

# def fuzzy_merge(df_1, df_2, key1, key2, threshold=92, limit=3):
#     """
#     :param df_1: the left table to join
#     :param df_2: the right table to join
#     :param key1: key column of the left table
#     :param key2: key column of the right table
#     :param threshold: how close the matches should be to return a match, based on Levenshtein distance
#     :param limit: the amount of matches that will get returned, these are sorted high to low
#     :return: dataframe with boths keys and matches
#     """
    
#     s = df_2[key2].tolist()
    
#     m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
#     df_1['matches'] = m
#     m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
#     m2_score = df_1['matches'].apply(lambda x: ', '.join([str(i[1]) for i in x if i[1] >= threshold]))
#     m2_BoardEx_raw = df_1['matches'].apply(lambda x: ', '.join([BoardEx_nameid_dic[i[0]][0] for i in x if i[1] >= threshold]))
#     m2_companyid = df_1['matches'].apply(lambda x: ', '.join([str(BoardEx_nameid_dic[i[0]][1]) for i in x if i[1] >= threshold]))
#     df_1['matches'] = m2
#     df_1['scores'] = m2_score
#     df_1['BoardEx_raw'] = m2_BoardEx_raw
#     df_1['companyid_of_match'] = m2_companyid
#     df_1 = df_1[['BoardExname','companyid','clean_name','Nielsenname','prefix','matches','scores','BoardEx_raw','companyid_of_match']]
#     print(str(len(df_1))+" more firms in Nielsen have been processed.")

#     return df_1

# def proc_list(Nielsen_not_merged):

#     return fuzzy_merge(Nielsen_not_merged, BoardEx_nameid[:], "clean_name", "clean_name")

In [17]:
# %%time

# BoardEx_nameid_dic = BoardEx_nameid.set_index('clean_name').T.to_dict("list")

# Nielsen_not_merged_dd = dd.from_pandas(Nielsen_not_merged[:], npartitions=40)
# with dask.config.set(scheduler='processes'):
#     Fuzzy_matched = Nielsen_not_merged_dd.map_partitions(proc_list, meta=pd.DataFrame(columns=['BoardExname','companyid','clean_name','Nielsenname','prefix','matches','scores','BoardEx_raw','companyid_of_match'])).compute()


In [18]:
# # Mannually inspect results from fuzzy matching

# Fuzzy_matched = Fuzzy_matched[Fuzzy_matched["matches"]!=""]

# Fuzzy_matched.to_csv("Fuzzy_matched.csv")

### Results of hand-matching

In [19]:
# A list of matches from manual inspection
# Record prefix and companyid first, and then merge in other columns

# These are matches found by Renping, which are the first 1/6

Matches = []
Matches = Matches+[{"prefix":"031465","companyid":1671165.0}]
Matches = Matches+[{"prefix":"018200","companyid":855900.0}]
Matches = Matches+[{"prefix":"076072","companyid":855900.0}]
Matches = Matches+[{"prefix":"088753","companyid":855900.0}]
Matches = Matches+[{"prefix":"0018200","companyid":855900.0}]
Matches = Matches+[{"prefix":"0076072","companyid":855900.0}]
Matches = Matches+[{"prefix":"071179","companyid":117291.0}]
Matches = Matches+[{"prefix":"0071179","companyid":117291.0}]
Matches = Matches+[{"prefix":"071409","companyid":1235935.0}]
Matches = Matches+[{"prefix":"0071409","companyid":1235935.0}]
Matches = Matches+[{"prefix":"071596","companyid":261947.0}]
Matches = Matches+[{"prefix":"0071596","companyid":261947.0}]
Matches = Matches+[{"prefix":"071670","companyid":261947.0}]
Matches = Matches+[{"prefix":"0071670","companyid":261947.0}]
Matches = Matches+[{"prefix":"009468","companyid":261947.0}]
Matches = Matches+[{"prefix":"009450","companyid":261947.0}]
Matches = Matches+[{"prefix":"073520","companyid":2604765.0}]
Matches = Matches+[{"prefix":"075372","companyid":270982.0}]
Matches = Matches+[{"prefix":"0043646","companyid":270982.0}]
Matches = Matches+[{"prefix":"0075372","companyid":270982.0}]
Matches = Matches+[{"prefix":"075324","companyid":743430.0}]
Matches = Matches+[{"prefix":"0075324","companyid":743430.0}]
Matches = Matches+[{"prefix":"075828","companyid":564589.0}]
Matches = Matches+[{"prefix":"0075828","companyid":564589.0}]
Matches = Matches+[{"prefix":"075919","companyid":2078239.0}]
Matches = Matches+[{"prefix":"074856","companyid":2078239.0}]
Matches = Matches+[{"prefix":"0030192","companyid":2078239.0}]
Matches = Matches+[{"prefix":"0026452","companyid":2078239.0}]
Matches = Matches+[{"prefix":"0074856","companyid":2078239.0}]
Matches = Matches+[{"prefix":"0075919","companyid":2078239.0}]
Matches = Matches+[{"prefix":"078638","companyid":1513466.0}]
Matches = Matches+[{"prefix":"079113","companyid":2724900.0}]
Matches = Matches+[{"prefix":"0079113","companyid":2724900.0}]
Matches = Matches+[{"prefix":"076166","companyid":1545146.0}]
Matches = Matches+[{"prefix":"070510","companyid":810958.0}]
Matches = Matches+[{"prefix":"0070510","companyid":810958.0}]
Matches = Matches+[{"prefix":"0072238","companyid":27003.0}]
Matches = Matches+[{"prefix":"0072261","companyid":27003.0}]
Matches = Matches+[{"prefix":"0070690","companyid":27003.0}]
Matches = Matches+[{"prefix":"0041676","companyid":27003.0}]
Matches = Matches+[{"prefix":"072238","companyid":27003.0}]
Matches = Matches+[{"prefix":"085839","companyid":27003.0}]
Matches = Matches+[{"prefix":"070690","companyid":27003.0}]
Matches = Matches+[{"prefix":"070674","companyid":633392.0}]
Matches = Matches+[{"prefix":"073608","companyid":2204232.0}]
Matches = Matches+[{"prefix":"0073608","companyid":2204232.0}]
Matches = Matches+[{"prefix":"076241","companyid":1293333.0}]
Matches = Matches+[{"prefix":"0076301","companyid":827247.0}]
Matches = Matches+[{"prefix":"0046700","companyid":827247.0}]
Matches = Matches+[{"prefix":"076301","companyid":827247.0}]
Matches = Matches+[{"prefix":"0076366","companyid":1820920.0}]
Matches = Matches+[{"prefix":"0073202","companyid":1820920.0}]
Matches = Matches+[{"prefix":"0070077","companyid":1820920.0}]
Matches = Matches+[{"prefix":"0050665","companyid":1820920.0}]
Matches = Matches+[{"prefix":"0010878","companyid":1820920.0}]
Matches = Matches+[{"prefix":"070077","companyid":1820920.0}]
Matches = Matches+[{"prefix":"076366","companyid":1820920.0}]
Matches = Matches+[{"prefix":"075512","companyid":1983908.0}]
Matches = Matches+[{"prefix":"074184","companyid":1986720.0}]
Matches = Matches+[{"prefix":"074208","companyid":2201292.0}]
Matches = Matches+[{"prefix":"0074208","companyid":2201292.0}]
Matches = Matches+[{"prefix":"072101","companyid":423385.0}]
Matches = Matches+[{"prefix":"0072101","companyid":423385.0}]
Matches = Matches+[{"prefix":"072109","companyid":1975109.0}]
Matches = Matches+[{"prefix":"0072109","companyid":1975109.0}]
Matches = Matches+[{"prefix":"074297","companyid":2035577.0}]
Matches = Matches+[{"prefix":"0070980","companyid":2143144.0}]
Matches = Matches+[{"prefix":"070980","companyid":2143144.0}]
Matches = Matches+[{"prefix":"072249","companyid":2760815.0}]
Matches = Matches+[{"prefix":"0072249","companyid":2760815.0}]
Matches = Matches+[{"prefix":"071075","companyid":1347377.0}]
Matches = Matches+[{"prefix":"0077769","companyid":1685436.0}]
Matches = Matches+[{"prefix":"077769","companyid":1685436.0}]
Matches = Matches+[{"prefix":"074567","companyid":1628730.0}]
Matches = Matches+[{"prefix":"0080716","companyid":2522588.0}]
Matches = Matches+[{"prefix":"080716","companyid":2522588.0}]
Matches = Matches+[{"prefix":"078119","companyid":2522588.0}]


In [20]:
# These are matches found by Meishu, which are the rest 5/6

Matches = Matches+[{"prefix":"074988","companyid":105872.0}]
Matches = Matches+[{"prefix":"072656","companyid":1670312.0}]
Matches = Matches+[{"prefix":"0038058","companyid":1670312.0}]
Matches = Matches+[{"prefix":"0072656","companyid":1670312.0}]
Matches = Matches+[{"prefix":"075021","companyid":731099.0}]
Matches = Matches+[{"prefix":"083603","companyid":731099.0}]
Matches = Matches+[{"prefix":"0075021","companyid":731099.0}]
Matches = Matches+[{"prefix":"082616","companyid":1548089.0}]
Matches = Matches+[{"prefix":"075106","companyid":1907560.0}]
Matches = Matches+[{"prefix":"0075106","companyid":1907560.0}]
Matches = Matches+[{"prefix":"073214","companyid":2769778.0}]
Matches = Matches+[{"prefix":"0073214","companyid":2769778.0}]
Matches = Matches+[{"prefix":"073228","companyid":81853.0}]
Matches = Matches+[{"prefix":"0073228","companyid":81853.0}]
Matches = Matches+[{"prefix":"082879","companyid":1020271.0}]
Matches = Matches+[{"prefix":"083089","companyid":1693579.0}]
Matches = Matches+[{"prefix":"750105","companyid":1056422.0}]
Matches = Matches+[{"prefix":"759101","companyid":757904.0}]
Matches = Matches+[{"prefix":"800738","companyid":1619482.0}]
Matches = Matches+[{"prefix":"888819","companyid":1570739.0}]
Matches = Matches+[{"prefix":"083417","companyid":1961379.0}]
Matches = Matches+[{"prefix":"085000","companyid":69562.0}]
Matches = Matches+[{"prefix":"085436","companyid":69562.0}]
Matches = Matches+[{"prefix":"085800","companyid":69562.0}]
Matches = Matches+[{"prefix":"0018341","companyid":69562.0}]
Matches = Matches+[{"prefix":"0015904","companyid":69562.0}]
Matches = Matches+[{"prefix":"085127","companyid":1211748.0}]
Matches = Matches+[{"prefix":"083567","companyid":2564901.0}]
Matches = Matches+[{"prefix":"085266","companyid":2051959.0}]
Matches = Matches+[{"prefix":"085277","companyid":2295409.0}]
Matches = Matches+[{"prefix":"085763","companyid":2018267.0}]
Matches = Matches+[{"prefix":"083592","companyid":1352378.0}]
Matches = Matches+[{"prefix":"087625","companyid":860194.0}]
Matches = Matches+[{"prefix":"0014001","companyid":860194.0}]
Matches = Matches+[{"prefix":"087643","companyid":242962.0}]
Matches = Matches+[{"prefix":"0021399","companyid":242962.0}]
Matches = Matches+[{"prefix":"087769","companyid":742322.0}]
Matches = Matches+[{"prefix":"087901","companyid":2147299.0}]
Matches = Matches+[{"prefix":"084124","companyid":2065316.0}]
Matches = Matches+[{"prefix":"084227","companyid":1381622.0}]
Matches = Matches+[{"prefix":"084296","companyid":818793.0}]
Matches = Matches+[{"prefix":"711027","companyid":818793.0}]
Matches = Matches+[{"prefix":"082087","companyid":818793.0}]
Matches = Matches+[{"prefix":"783722","companyid":818793.0}]
Matches = Matches+[{"prefix":"088105","companyid":7293.0}]
Matches = Matches+[{"prefix":"089112","companyid":647072.0}]
Matches = Matches+[{"prefix":"089105","companyid":2394692.0}]
Matches = Matches+[{"prefix":"089182","companyid":2696485.0}]
Matches = Matches+[{"prefix":"089552","companyid":625540.0}]
Matches = Matches+[{"prefix":"089541","companyid":1635441.0}]
Matches = Matches+[{"prefix":"009192","companyid":1326512.0}]
Matches = Matches+[{"prefix":"009285","companyid":1064823.0}]
Matches = Matches+[{"prefix":"018529","companyid":2815392.0}]
Matches = Matches+[{"prefix":"0018529","companyid":2815392.0}]
Matches = Matches+[{"prefix":"018646","companyid":2398276.0}]
Matches = Matches+[{"prefix":"0018646","companyid":2398276.0}]
Matches = Matches+[{"prefix":"077854","companyid":77572.0}]
Matches = Matches+[{"prefix":"078092","companyid":17216.0}]
Matches = Matches+[{"prefix":"073321","companyid":17216.0}]
Matches = Matches+[{"prefix":"0020593","companyid":17216.0}]
Matches = Matches+[{"prefix":"0014821","companyid":17216.0}]
Matches = Matches+[{"prefix":"0657243","companyid":17216.0}]
Matches = Matches+[{"prefix":"0669731","companyid":17216.0}]
Matches = Matches+[{"prefix":"0073321","companyid":17216.0}]
Matches = Matches+[{"prefix":"0078092","companyid":17216.0}]
Matches = Matches+[{"prefix":"074345","companyid":626293.0}]
Matches = Matches+[{"prefix":"0074345","companyid":626293.0}]
Matches = Matches+[{"prefix":"070820","companyid":1718876.0}]
Matches = Matches+[{"prefix":"0070820","companyid":1718876.0}]
Matches = Matches+[{"prefix":"691063","companyid":1514007.0}]
Matches = Matches+[{"prefix":"745751","companyid":1975637.0}]
Matches = Matches+[{"prefix":"746417","companyid":19404.0}]
Matches = Matches+[{"prefix":"08409030","companyid":19404.0}]
Matches = Matches+[{"prefix":"079111","companyid":621437.0}]
Matches = Matches+[{"prefix":"0079111","companyid":621437.0}]
Matches = Matches+[{"prefix":"789893","companyid":731686.0}]
Matches = Matches+[{"prefix":"088374","companyid":2523504.0}]
Matches = Matches+[{"prefix":"031514","companyid":631043.0}]
Matches = Matches+[{"prefix":"074010","companyid":1807778.0}]
Matches = Matches+[{"prefix":"0074010","companyid":1807778.0}]
Matches = Matches+[{"prefix":"750611","companyid":2715463.0}]
Matches = Matches+[{"prefix":"800461","companyid":1071431.0}]
Matches = Matches+[{"prefix":"885536","companyid":1007211.0}]
Matches = Matches+[{"prefix":"733003","companyid":802581.0}]
Matches = Matches+[{"prefix":"0636202","companyid":802581.0}]
Matches = Matches+[{"prefix":"801869","companyid":742887.0}]
Matches = Matches+[{"prefix":"080868","companyid":2021306.0}]
Matches = Matches+[{"prefix":"0080868","companyid":2021306.0}]
Matches = Matches+[{"prefix":"088682","companyid":783723.0}]
Matches = Matches+[{"prefix":"0024600","companyid":55481.0}]
Matches = Matches+[{"prefix":"0010164","companyid":2077992.0}]
Matches = Matches+[{"prefix":"0020932","companyid":2514339.0}]
Matches = Matches+[{"prefix":"0010594","companyid":2133852.0}]
Matches = Matches+[{"prefix":"0016236","companyid":606432.0}]
Matches = Matches+[{"prefix":"0021015","companyid":633872.0}]
Matches = Matches+[{"prefix":"0021120","companyid":641308.0}]
Matches = Matches+[{"prefix":"0029600","companyid":482301.0}]
Matches = Matches+[{"prefix":"0023545","companyid":2318798.0}]
Matches = Matches+[{"prefix":"0030003","companyid":607195.0}]
Matches = Matches+[{"prefix":"030003","companyid":607195.0}]
Matches = Matches+[{"prefix":"0012842","companyid":894615.0}]
Matches = Matches+[{"prefix":"0012885","companyid":1470843.0}]
Matches = Matches+[{"prefix":"0013024","companyid":1233422.0}]
Matches = Matches+[{"prefix":"0015755","companyid":85357.0}]
Matches = Matches+[{"prefix":"0015800","companyid":921186.0}]
Matches = Matches+[{"prefix":"0033604","companyid":740583.0}]
Matches = Matches+[{"prefix":"0025215","companyid":1822284.0}]
Matches = Matches+[{"prefix":"0034197","companyid":1227491.0}]
Matches = Matches+[{"prefix":"0034228","companyid":606006.0}]
Matches = Matches+[{"prefix":"0027506","companyid":2205664.0}]
Matches = Matches+[{"prefix":"0031000","companyid":1653515.0}]
Matches = Matches+[{"prefix":"0048300","companyid":1653515.0}]
Matches = Matches+[{"prefix":"0034742","companyid":664765.0}]
Matches = Matches+[{"prefix":"0034885","companyid":2539406.0}]
Matches = Matches+[{"prefix":"0025586","companyid":1818837.0}]
Matches = Matches+[{"prefix":"0021557","companyid":2000138.0}]
Matches = Matches+[{"prefix":"0035336","companyid":817262.0}]
Matches = Matches+[{"prefix":"0042421","companyid":1944381.0}]
Matches = Matches+[{"prefix":"0042493","companyid":1234579.0}]
Matches = Matches+[{"prefix":"0045007","companyid":881194.0}]
Matches = Matches+[{"prefix":"0045404","companyid":564376.0}]
Matches = Matches+[{"prefix":"0076370","companyid":564376.0}]
Matches = Matches+[{"prefix":"0042804","companyid":2778482.0}]
Matches = Matches+[{"prefix":"0042741","companyid":31878.0}]
Matches = Matches+[{"prefix":"0098811","companyid":1876714.0}]
Matches = Matches+[{"prefix":"0642193","companyid":492249.0}]
Matches = Matches+[{"prefix":"0619251","companyid":17404.0}]
Matches = Matches+[{"prefix":"0647736","companyid":106692.0}]
Matches = Matches+[{"prefix":"0650601","companyid":254967.0}]
Matches = Matches+[{"prefix":"0652665","companyid":2530337.0}]
Matches = Matches+[{"prefix":"0654050","companyid":466862.0}]
Matches = Matches+[{"prefix":"0653621","companyid":587226.0}]
Matches = Matches+[{"prefix":"0630557","companyid":2208149.0}]
Matches = Matches+[{"prefix":"0654700","companyid":1015836.0}]
Matches = Matches+[{"prefix":"0634528","companyid":2288898.0}]
Matches = Matches+[{"prefix":"0031134","companyid":1710729.0}]
Matches = Matches+[{"prefix":"0660089","companyid":32014.0}]
Matches = Matches+[{"prefix":"0635753","companyid":639098.0}]
Matches = Matches+[{"prefix":"0047884","companyid":2535312.0}]
Matches = Matches+[{"prefix":"0635824","companyid":2530780.0}]
Matches = Matches+[{"prefix":"0607342","companyid":1057290.0}]
Matches = Matches+[{"prefix":"0661168","companyid":1670740.0}]
Matches = Matches+[{"prefix":"0637295","companyid":463708.0}]
Matches = Matches+[{"prefix":"0046504","companyid":1210032.0}]
Matches = Matches+[{"prefix":"0667721","companyid":2722770.0}]
Matches = Matches+[{"prefix":"0687926","companyid":735292.0}]
Matches = Matches+[{"prefix":"0609193","companyid":2019460.0}]
Matches = Matches+[{"prefix":"0670984","companyid":107522.0}]
Matches = Matches+[{"prefix":"0043732","companyid":897701.0}]
Matches = Matches+[{"prefix":"0609961","companyid":1652267.0}]
Matches = Matches+[{"prefix":"0602755","companyid":1664755.0}]
Matches = Matches+[{"prefix":"0611167","companyid":934978.0}]
Matches = Matches+[{"prefix":"0689839","companyid":856623.0}]
Matches = Matches+[{"prefix":"0689192","companyid":1975583.0}]
Matches = Matches+[{"prefix":"0032017","companyid":68580.0}]
Matches = Matches+[{"prefix":"0682906","companyid":2632158.0}]
Matches = Matches+[{"prefix":"0686776","companyid":1246496.0}]
Matches = Matches+[{"prefix":"0686114","companyid":2300913.0}]
Matches = Matches+[{"prefix":"0051200","companyid":2319986.0}]
Matches = Matches+[{"prefix":"0051900","companyid":105992.0}]
Matches = Matches+[{"prefix":"0075094","companyid":105992.0}]
Matches = Matches+[{"prefix":"0052373","companyid":2627140.0}]
Matches = Matches+[{"prefix":"0750659","companyid":2031998.0}]
Matches = Matches+[{"prefix":"0052717","companyid":100751.0}]
Matches = Matches+[{"prefix":"0730188","companyid":886306.0}]
Matches = Matches+[{"prefix":"0036725","companyid":744985.0}]
Matches = Matches+[{"prefix":"0066296","companyid":2723294.0}]
Matches = Matches+[{"prefix":"0038698","companyid":2723294.0}]
Matches = Matches+[{"prefix":"0040295","companyid":2013255.0}]
Matches = Matches+[{"prefix":"0040819","companyid":2624398.0}]
Matches = Matches+[{"prefix":"0037529","companyid":639077.0}]
Matches = Matches+[{"prefix":"0041224","companyid":2033164.0}]
Matches = Matches+[{"prefix":"0041433","companyid":2306830.0}]
Matches = Matches+[{"prefix":"0075607","companyid":816326.0}]
Matches = Matches+[{"prefix":"0072211","companyid":423595.0}]
Matches = Matches+[{"prefix":"0881016","companyid":851247.0}]
Matches = Matches+[{"prefix":"0041318","companyid":915475.0}]
Matches = Matches+[{"prefix":"0039639","companyid":464295.0}]
Matches = Matches+[{"prefix":"0040159","companyid":1902497.0}]
Matches = Matches+[{"prefix":"0079746","companyid":2473868.0}]
Matches = Matches+[{"prefix":"0019648","companyid":2140799.0}]
Matches = Matches+[{"prefix":"0027011","companyid":1201597.0}]
Matches = Matches+[{"prefix":"0022035","companyid":1294.0}]
Matches = Matches+[{"prefix":"0013431","companyid":860492.0}]
Matches = Matches+[{"prefix":"0095587","companyid":7293.0}]
Matches = Matches+[{"prefix":"0024964","companyid":1679971.0}]
Matches = Matches+[{"prefix":"0013789","companyid":1686722.0}]
Matches = Matches+[{"prefix":"0033995","companyid":1541020.0}]
Matches = Matches+[{"prefix":"0030238","companyid":2375814.0}]
Matches = Matches+[{"prefix":"0014666","companyid":7293.0}]
Matches = Matches+[{"prefix":"0049353","companyid":2386922.0}]
Matches = Matches+[{"prefix":"0036357","companyid":1797941.0}]
Matches = Matches+[{"prefix":"0050937","companyid":1476292.0}]
Matches = Matches+[{"prefix":"0041802","companyid":1586110.0}]
Matches = Matches+[{"prefix":"0076043","companyid":1284984.0}]
Matches = Matches+[{"prefix":"0614130","companyid":1717056.0}]
Matches = Matches+[{"prefix":"0674183","companyid":1160.0}]
Matches = Matches+[{"prefix":"0640113","companyid":2723553.0}]
Matches = Matches+[{"prefix":"0683211","companyid":1194926.0}]
Matches = Matches+[{"prefix":"0641817","companyid":20749.0}]
Matches = Matches+[{"prefix":"0646192","companyid":870345.0}]
Matches = Matches+[{"prefix":"0791202","companyid":1798152.0}]
Matches = Matches+[{"prefix":"0047659","companyid":2010983.0}]
Matches = Matches+[{"prefix":"0635093","companyid":2202887.0}]
Matches = Matches+[{"prefix":"0667456","companyid":828283.0}]
Matches = Matches+[{"prefix":"0086382","companyid":1378884.0}]
Matches = Matches+[{"prefix":"0608641","companyid":1877492.0}]
Matches = Matches+[{"prefix":"0099904","companyid":1651568.0}]
Matches = Matches+[{"prefix":"0633356","companyid":2217324.0}]
Matches = Matches+[{"prefix":"81396501","companyid":667781.0}]
Matches = Matches+[{"prefix":"087430500","companyid":1071750.0}]
Matches = Matches+[{"prefix":"087910500","companyid":2345539.0}]
Matches = Matches+[{"prefix":"081370101","companyid":2179167.0}]
Matches = Matches+[{"prefix":"30256","companyid":1746869.0}]
Matches = Matches+[{"prefix":"38076","companyid":17478.0}]
Matches = Matches+[{"prefix":"30038","companyid":894072.0}]
Matches = Matches+[{"prefix":"030078","companyid":600069.0}]
Matches = Matches+[{"prefix":"30028","companyid":563135.0}]


In [21]:
Merged_fuzzy = pd.DataFrame(Matches)
Merged_fuzzy = pd.merge(Merged_fuzzy, BoardEx_nameid.drop_duplicates('companyid'), on="companyid", how="inner")
Merged_fuzzy = pd.merge(Merged_fuzzy, Nielsen_nameprefix, on="prefix", how="inner")

## 6. Manually match Top 500 prefixes with most revenue

In [22]:
# After doing all the matches above, I sort all prefixes by total revenue over the sample period.
# Then I focus on the 500 with most revenue (in rev_n_prefix_09092022.csv), and look at ones that are not matched to BoardEx yet.
# Then I come back here, and mannually match those unmatched ones.

# use "/Users/renpingli/Dropbox/DirectorNetwork/RawData/boardexemployment.dta", clear
# bysort companyid : egen dircount = count(directorid )
# duplicates drop companyid , force
# gsort -dircount +companyname 
# order dircount companyname
# Then manually look for match

Matches = []
Matches = Matches+[{"prefix":"0028200","companyid":78221.0}]
Matches = Matches+[{"prefix":"0049000","companyid":7204.0}]
Matches = Matches+[{"prefix":"0036000","companyid":17913.0}]
Matches = Matches+[{"prefix":"0013800","companyid":791643.0}]
Matches = Matches+[{"prefix":"0052000","companyid":25526.0}]
Matches = Matches+[{"prefix":"0036632","companyid":8678.0}]
Matches = Matches+[{"prefix":"0042000","companyid":12310.0}]
Matches = Matches+[{"prefix":"0048001","companyid":270982.0}]
Matches = Matches+[{"prefix":"0041100","companyid":95202.0}]
Matches = Matches+[{"prefix":"0024300","companyid":624944.0}]
Matches = Matches+[{"prefix":"0012587","companyid":40888.0}]
Matches = Matches+[{"prefix":"0036200","companyid":2205373.0}]
Matches = Matches+[{"prefix":"0021131","companyid":836667.0}]
Matches = Matches+[{"prefix":"0041271","companyid":462254.0}]
Matches = Matches+[{"prefix":"0072890","companyid":816255.0}]
Matches = Matches+[{"prefix":"0025293","companyid":8678.0}]
Matches = Matches+[{"prefix":"0016500","companyid":95202.0}]
Matches = Matches+[{"prefix":"0076808","companyid":1102096.0}]
Matches = Matches+[{"prefix":"0087000","companyid":27413.0}]
Matches = Matches+[{"prefix":"008660","companyid":4761.0}]
Matches = Matches+[{"prefix":"0043168","companyid":1653367.0}]
Matches = Matches+[{"prefix":"008858","companyid":786063.0}]
Matches = Matches+[{"prefix":"0075278","companyid":272287.0}]
Matches = Matches+[{"prefix":"0075020","companyid":740482.0}]
Matches = Matches+[{"prefix":"0043396","companyid":1813130.0}]
Matches = Matches+[{"prefix":"0071691","companyid":49145.0}]
Matches = Matches+[{"prefix":"0041900","companyid":462254.0}]
Matches = Matches+[{"prefix":"0052092","companyid":2145350.0}]
Matches = Matches+[{"prefix":"0037466","companyid":1940987.0}]
Matches = Matches+[{"prefix":"0025192","companyid":2556909.0}]
Matches = Matches+[{"prefix":"0015300","companyid":25526.0}]
Matches = Matches+[{"prefix":"0071007","companyid":141592.0}]
Matches = Matches+[{"prefix":"0011017","companyid":95202.0}]
Matches = Matches+[{"prefix":"0093966","companyid":1977145.0}]
Matches = Matches+[{"prefix":"0026200","companyid":7619.0}]
Matches = Matches+[{"prefix":"0037700","companyid":1487661.0}]
Matches = Matches+[{"prefix":"0072311","companyid":834161.0}]
Matches = Matches+[{"prefix":"0022796","companyid":2233067.0}]
Matches = Matches+[{"prefix":"0075243","companyid":462254.0}]
Matches = Matches+[{"prefix":"0041790","companyid":28614.0}]
Matches = Matches+[{"prefix":"0027242","companyid":28594.0}]
Matches = Matches+[{"prefix":"0072322","companyid":58110.0}]
Matches = Matches+[{"prefix":"071683","companyid":1482432.0}]
Matches = Matches+[{"prefix":"0070100","companyid":28311.0}]
Matches = Matches+[{"prefix":"0046034","companyid":26617.0}]
Matches = Matches+[{"prefix":"0013803","companyid":5671.0}]
Matches = Matches+[{"prefix":"0027815","companyid":28311.0}]
Matches = Matches+[{"prefix":"0077802","companyid":2361250.0}]

Merged_toprev = pd.DataFrame(Matches)
Merged_toprev = pd.merge(Merged_toprev, BoardEx_nameid.drop_duplicates('companyid'), on="companyid", how="inner")
Merged_toprev = pd.merge(Merged_toprev, Nielsen_nameprefix, on="prefix", how="inner")


## 7. Manual correction

In [23]:
# Merge together parts that have already been processed

Merged = pd.concat([Merged_usingparent,Merged_usingsubsidiary,Merged_fuzzy,Merged_toprev])
Merged = Merged.sort_values(['Nielsenname'])

In [24]:
# Manually add correction and delete incorrect counterparts

Matches = []

# "Sjm Corporation" in GS1 refers to The J.M. Smucker Company not SJM Holdings
Matches = Matches+[{"prefix":"000838","companyid":28325.0}]
Merged = Merged[Merged["prefix"]!="000838"]

# "Ms. Foods Inc" should not be matched to Morgan Stanley
Merged = Merged[Merged["prefix"]!="0748389"]

Merged_manualcorr = pd.DataFrame(Matches)
Merged_manualcorr = pd.merge(Merged_manualcorr, BoardEx_nameid.drop_duplicates('companyid'), on="companyid", how="inner")
Merged_manualcorr = pd.merge(Merged_manualcorr, Nielsen_nameprefix, on="prefix", how="inner")


## 8. Export the merged link file

In [25]:
# Merge in the correction

Merged = pd.concat([Merged,Merged_manualcorr])
Merged = Merged.sort_values(['Nielsenname'])


In [45]:
Merged['Occur'] = Merged.groupby('prefix')['prefix'].transform('size')
print('There are '+str(Merged.value_counts('Occur')[2])+' cases of duplicates, which happens because of the manual match')
print('step. I randomly keep one.')
Merged = Merged.drop_duplicates('prefix')

In [49]:
Merged = Merged[['Nielsenname','prefix','BoardExname','companyid']].reset_index()
Merged.to_csv("../CleanedData/BoardExNielsen_link.csv")